# Clean Data Example
An example to show how to use regular expressions to clean data in an OSM file, an OpenStreetMap data file downloaded from [Mapzen](https://mapzen.com/data/metro-extracts/).

In [ ]:
from zipfile import ZipFile
import xml.etree.ElementTree as ET
from collections import defaultdict
import re

filename = 'southampton_england.osm.zip'

# # Create a function that opens the zip file.
# # Extract all members from the archive to the current working directory.
# def extract_zip(fname):
#     with ZipFile(fname, 'r') as myzip:
#         myzip.extractall()
# extract_zip(filename)
# osm_file = 'southampton_england.osm'

# Open the zipped file.
with ZipFile(filename, 'r') as zfile:
    unzippedfile = zfile.open('southampton_england.osm')

osm_file = unzippedfile

# Use re.compile() and save the resulting regular expression object for reuse.
street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)

# Create a defaultdict (a normal dict, but will never raise a KeyError for a nonexistent key).
street_types = defaultdict(int)

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        street_types[street_type] += 1

def print_sorted_dict(d):
    keys = d.keys()
    # Use lambda function to specify lower() directly, inline in the sorted() expression.
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print("%s: %d" % (k, v)) 

def is_street_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:street")

def audit():
    # Parse the XML section into element tree incrementally. Useful for large files.
    for event, elem in ET.iterparse(osm_file):
        if is_street_name(elem):
            audit_street_type(street_types, elem.attrib['v'])    
    print_sorted_dict(street_types)    

if __name__ == '__main__':
    audit()